In [1]:
import pandas as pd
import pyarrow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
pd.__version__

'2.3.0'

In [3]:
df=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
# Show the number of columns in the dataset
print(f"Number of columns: {df.shape[1]}")

Number of columns: 19


In [6]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculate standard deviation of duration
duration_std = df['duration'].std()
print(f"Standard deviation of ride duration: {duration_std:.2f} minutes")

Standard deviation of ride duration: 42.59 minutes


In [7]:
print(f"Number of rows before filtering: {len(df)}")

# Remove outliers - use query for better memory efficiency
df = df.query('duration >= 1 and duration <= 60').copy()

# Recalculate standard deviation after removing outliers
duration_std = df['duration'].std()
print(f"Standard deviation of ride duration after removing outliers: {duration_std:.2f} minutes")
print(f"Number of rows after filtering: {len(df)}")

Number of rows before filtering: 3066766


Standard deviation of ride duration after removing outliers: 9.94 minutes
Number of rows after filtering: 3009173


In [8]:
# Q4. One-hot encoding
# Turn the dataframe into a list of dictionaries for pickup and dropoff location IDs
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

# Convert to list of dictionaries
train_dicts = df[categorical].to_dict(orient='records')

# Fit DictVectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f"Q4. Dimensionality of feature matrix: {X_train.shape[1]}")

Q4. Dimensionality of feature matrix: 515


In [9]:
# Q5. Training a model
# Train linear regression model with duration as target
y_train = df['duration'].values

# Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Calculate RMSE on training data
import numpy as np
y_pred_train = lr.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"Q5. RMSE on training data: {rmse_train:.2f}")

Q5. RMSE on training data: 7.65


In [10]:
# Q6. Evaluating the model
# Load validation dataset (February 2023)
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

# Apply same preprocessing
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60
df_val = df_val.query('duration >= 1 and duration <= 60').copy()

# Prepare validation features
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)  # Use transform, not fit_transform
y_val = df_val['duration'].values

# Calculate RMSE on validation data
y_pred_val = lr.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Q6. RMSE on validation data: {rmse_val:.2f}")

Q6. RMSE on validation data: 7.81


In [11]:
# Save the model and vectorizer together
import pickle
import os

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save both the DictVectorizer and LinearRegression model as a tuple
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

print("Model and vectorizer saved successfully!")
print("File created: models/lin_reg.bin")

Model and vectorizer saved successfully!
File created: models/lin_reg.bin
